In [6]:
!pip install cx_Oracle

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build cx_Oracle


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [21 lines of output]
  <string>:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  C:\Users\pietr\AppData\Local\Temp\pip-build-env-_11tmlts\overlay\Lib\site-packages\setuptools\config\expand.py:126: SetuptoolsWarning: File 'C:\\Users\\pietr\\AppData\\Local\\Temp\\pip-install-kekq2pbr\\cx-oracle_add3cd6750b04bc6ad6731ac52f1b048\\README.md' cannot be found
    return '\n'.join(
  C:\Users\pietr\AppData\Local\Temp\pip-build-env-_11tmlts\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved :: BSD License
  
          See https://packaging.python.org/en/latest/guide

In [5]:
import cv2
import time
from ultralytics import YOLO
import cx_Oracle
import sys
import os

# === CONFIGURAÇÃO DO BANCO ORACLE ===
try:
    dsn = cx_Oracle.makedsn("oracle.fiap.com.br", 1521, service_name="orcl")
    conn = cx_Oracle.connect(
        user="seu-user",
        password="sua-senha",
        dsn=dsn
    )
    print("Conectado ao banco com sucesso!")
    cursor = conn.cursor()
except cx_Oracle.DatabaseError as e:
    print("Erro ao conectar ao banco de dados Oracle:", e)
    sys.exit(1)

# === FUNÇÃO DE INSERÇÃO/ATUALIZAÇÃO ===
def salvar_ou_atualizar_no_banco(dado):
    try:
        sql = """
        MERGE INTO MOTOS_LOCALIZADAS dest
        USING (
            SELECT :object_id AS object_id FROM dual
        ) src
        ON (dest.OBJECT_ID = src.object_id)
        WHEN MATCHED THEN
            UPDATE SET 
                TIMESTAMP_REG = TO_DATE(:timestamp, 'YYYY-MM-DD HH24:MI:SS'),
                CLASSE = :classe,
                POS_X = :pos_x,
                POS_Y = :pos_y,
                X1 = :x1,
                Y1 = :y1,
                X2 = :x2,
                Y2 = :y2
        WHEN NOT MATCHED THEN
            INSERT (OBJECT_ID, TIMESTAMP_REG, CLASSE, POS_X, POS_Y, X1, Y1, X2, Y2)
            VALUES (
                :object_id,
                TO_DATE(:timestamp, 'YYYY-MM-DD HH24:MI:SS'),
                :classe, :pos_x, :pos_y, :x1, :y1, :x2, :y2
            )
        """
        ts = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(dado["timestamp"]))
        cursor.execute(sql, {
            "object_id": dado["object_id"],
            "timestamp": ts,
            "classe": dado["class"],
            "pos_x": dado["position"]["x"],
            "pos_y": dado["position"]["y"],
            "x1": dado["bbox"]["x1"],
            "y1": dado["bbox"]["y1"],
            "x2": dado["bbox"]["x2"],
            "y2": dado["bbox"]["y2"]
        })
        conn.commit()
    except cx_Oracle.DatabaseError as e:
        print("Erro ao salvar dados no banco:", e)

# === FUNÇÃO PARA PROCESSAR UMA IMAGEM ===
def processar_imagem(imagem_path):
    imagem = cv2.imread(imagem_path)
    if imagem is None:
        print(f"Erro ao abrir imagem: {imagem_path}")
        return

    model = YOLO("yolov8s.pt")  # carregue uma vez fora do loop, se quiser processar várias

    results = model(imagem)
    boxes = results[0].boxes

    for box in boxes:
        cls_id = int(box.cls[0])
        class_name = model.names[cls_id]

        if class_name == "motorcycle":
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            obj_id = int(box.id[0]) if box.id is not None else None

            data = {
                "timestamp": time.time(),
                "object_id": obj_id,
                "class": class_name,
                "position": {"x": center_x, "y": center_y},
                "bbox": {"x1": x1, "y1": y1, "x2": x2, "y2": y2}
            }

            salvar_ou_atualizar_no_banco(data)
            print(f"[{class_name.upper()}] ID {obj_id} registrado com sucesso.")

    # Exibir a imagem com boxes
    imagem_com_boxes = results[0].plot()
    cv2.imshow("Detecção", imagem_com_boxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# === RODAR UM TESTE COM UMA IMAGEM ===
caminho_imagem = "caminho/para/sua/imagem.jpg"
processar_imagem(caminho_imagem)

# === ENCERRAR CONEXÃO ===
cursor.close()
conn.close()
print("Finalizado.")


ModuleNotFoundError: No module named 'cx_Oracle'